In [1]:
import tools.conf
import psycopg2
from psycopg2 import sql
# import psycopg
# from psycopg import sql
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./data/baza.csv', delimiter=';', thousands=',', dtype={'dł_ujednolicona': 'object'}, encoding='utf-8')
df.sort_values(by='data', axis=0, inplace=True)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)


In [3]:
df.head()

,data,dzien,dzien_tyg,nr_tyg,rok,miesiac,zasięg medium,brand,produkt(4),kod_reklamy,opis_reklamy,wydawca_nadawca,typ_reklamy,submedium,dl_mod,gg,mm,koszt,l_emisji,daypart,dł_ujednolicona,godzina_bloku_reklamowego
0,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194483,PATRZ BARBARA NO ALE..MEGA OKAZJE..SF SAMSUNG ...,SOME GROUP,reklama,SOME MINN Bydgoszcz,29,8,20,310,1,do 9,30,8:00-8:29
1,2023-08-01,1,Wtorek,31,2023,8,miejskie,MEDIA MASTER,GRUPA,22179830,MARIAN A GDZIE MOGĘ KUPIĆ..DOKUPIĆ..ŻEBY TO WS...,PIGLET S.A.,reklama,FROGGY WEATHER Warszawa,14,10,59,311,1,od 9 do 16,15,10:30-10:59
2,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194487,PATRZ BARBARA NO ALE..MEGA OKAZJE..TV SMART PH...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,30,15,22,73,1,od 9 do 16,30,15:00-15:29
3,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194445,PATRZ BARBARA NO ALE..MEGA OKAZJE..AUT. EKSPRE...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,29,14,42,67,1,od 9 do 16,30,14:30-14:59
4,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194475,BARBARA W - SĄ MEGA OKAZJE..MULTIRABATY..IM WI...,PIGLET S.A.,reklama,YELLOW HITS Zielona Góra,30,13,58,67,1,od 9 do 16,30,13:30-13:59


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366681 entries, 0 to 366680
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   data                       366681 non-null  object
 1   dzien                      366681 non-null  int64 
 2   dzien_tyg                  366681 non-null  object
 3   nr_tyg                     366681 non-null  int64 
 4   rok                        366681 non-null  int64 
 5   miesiac                    366681 non-null  int64 
 6   zasięg medium              366681 non-null  object
 7   brand                      366681 non-null  object
 8   produkt(4)                 366681 non-null  object
 9   kod_reklamy                366681 non-null  int64 
 10  opis_reklamy               366681 non-null  object
 11  wydawca_nadawca            366681 non-null  object
 12  typ_reklamy                366681 non-null  object
 13  submedium                  366681 non-null  

In [145]:
dow2 = ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek',
        'Sobota', 'Niedziela']

In [146]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


for day in dow2:
    cur.execute(
        sql.SQL('INSERT INTO {} (dow_name) VALUES (%s)')
        .format(sql.Identifier('pl_dow_names')), (day,)
    )

conn.commit()
conn.close()

In [147]:
months = ['Styczeń', 'Luty', 'Marzec', 'Kwiecień', 'Maj',
          'Czerwiec', 'Lipiec', 'Sierpień', 'Wrzesień',
          'Październik', 'Listopad', 'Grudzień']

In [148]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for month in months:
    cur.execute(
        query.format(
            table=sql.Identifier('pl_month_names'),
            field=sql.Identifier('month_name')), (month,)
    )

conn.commit()
conn.close()


In [149]:
dates = df['data'].unique()
dates

array(['2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
       '2023-08-05', '2023-08-06', '2023-08-07', '2023-08-08',
       '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-12',
       '2023-08-13', '2023-08-14', '2023-08-15', '2023-08-16',
       '2023-08-17', '2023-08-18', '2023-08-19', '2023-08-20',
       '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24',
       '2023-08-25', '2023-08-26', '2023-08-27', '2023-08-28',
       '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-01',
       '2023-09-02', '2023-09-03', '2023-09-04', '2023-09-05',
       '2023-09-06', '2023-09-07', '2023-09-08', '2023-09-09',
       '2023-09-10', '2023-09-11', '2023-09-12', '2023-09-13',
       '2023-09-14', '2023-09-15', '2023-09-16', '2023-09-17',
       '2023-09-18', '2023-09-19', '2023-09-20', '2023-09-21',
       '2023-09-22', '2023-09-23', '2023-09-24', '2023-09-25',
       '2023-09-26', '2023-09-27', '2023-09-28', '2023-09-29',
       '2023-09-30', '2023-10-01', '2023-10-02', '2023-

In [150]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()


query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for date in dates:
    cur.execute(
        query.format(
            table=sql.Identifier('date_time'),
            field=sql.Identifier('date')), (date,)
    )

conn.commit()
conn.close()

In [8]:
brands = df['brand'].sort_values().unique()
brands

array(['EURO APPLIANCES', 'MEDIA MASTER', 'MEDIA SHOP', 'NEWNET'],
      dtype=object)

In [10]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES(%s)')

for brand in brands:
    cur.execute(
        query.format(
            table=sql.Identifier('brands'),
            field=sql.Identifier('brand')), (brand,)
    )

conn.commit()
conn.close()

In [50]:
lengths = df['dł_ujednolicona'].sort_values().unique()
lengths

array(['15', '20', '30'], dtype=object)

In [51]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for length in lengths:
    cur.execute(
        query.format(
            table=sql.Identifier('unified_lenghts'),
            field=sql.Identifier('length')), (str(length),)
    )
    
conn.commit()
conn.close()

In [52]:
dayparts = df['daypart'].unique()
dayparts

array(['do 9', 'od 9 do 16', 'po 16'], dtype=object)

In [53]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for daypart in dayparts:
    cur.execute(
        query.format(
            table=sql.Identifier('dayparts'),
            field=sql.Identifier('daypart')), (str(daypart),)
    )
    
conn.commit()
conn.close()

In [22]:
product_types = df['produkt(4)'].sort_values().unique()
product_types

array(['AGD, RTV, ELEKTRONIKA, FOTOGRAFIA, KOMPUTERY', 'GRUPA',
       'OGŁOSZENIA O PRACY'], dtype=object)

In [23]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for product in product_types:
    cur.execute(
        query.format(
            table=sql.Identifier('product_types'),
            field=sql.Identifier('product_type')), (str(product),)
    )
    
conn.commit()
conn.close()

In [27]:
broadcasters = df['wydawca_nadawca'].sort_values().unique()
broadcasters

array(['BAR RADIO', 'EMKA RADIO', 'HOHOHO', 'KITTY RADIO', 'OLD RADIO',
       'OMG', 'PIANO RADIO', 'PIGLET S.A.', 'SAND', 'SO BORING RADIO',
       'SOME GROUP', 'VOLKSDONKY', 'ZZC GROUP'], dtype=object)

In [28]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for broadcaster in broadcasters:
    cur.execute(
        query.format(
            table=sql.Identifier('broadcasters'),
            field=sql.Identifier('broadcaster')), (str(broadcaster),)
    )
    
conn.commit()
conn.close()

In [30]:
reaches = df['zasięg medium'].unique()
reaches

array(['regionalne', 'miejskie', 'krajowe', 'ponadregionalne'],
      dtype=object)

In [31]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query = sql.SQL('INSERT INTO {table} ({field}) VALUES (%s)')

for reach in reaches:
    cur.execute(
        query.format(
            table=sql.Identifier('ad_reach'),
            field=sql.Identifier('reach')), (str(reach),)
    )
    
conn.commit()
conn.close()

In [36]:
df.head(2)

,data,dzien,dzien_tyg,nr_tyg,rok,miesiac,zasięg medium,brand,produkt(4),kod_reklamy,opis_reklamy,wydawca_nadawca,typ_reklamy,submedium,dl_mod,gg,mm,koszt,l_emisji,daypart,dł_ujednolicona,godzina_bloku_reklamowego
0,2023-08-01,1,Wtorek,31,2023,8,regionalne,MEDIA MASTER,GRUPA,22194483,PATRZ BARBARA NO ALE..MEGA OKAZJE..SF SAMSUNG ...,SOME GROUP,reklama,SOME MINN Bydgoszcz,29,8,20,310,1,do 9,30,8:00-8:29
1,2023-08-01,1,Wtorek,31,2023,8,miejskie,MEDIA MASTER,GRUPA,22179830,MARIAN A GDZIE MOGĘ KUPIĆ..DOKUPIĆ..ŻEBY TO WS...,PIGLET S.A.,reklama,FROGGY WEATHER Warszawa,14,10,59,311,1,od 9 do 16,15,10:30-10:59


In [124]:
submediums = df[['submedium', 'wydawca_nadawca', 'zasięg medium']].sort_values(by='submedium')
submediums.drop_duplicates(subset=['submedium'], keep='first', inplace=True, ignore_index=True)
submediums.index = submediums.index + 1
# submediums

In [125]:
sum(submediums.value_counts()) == submediums.index.max()

True

In [126]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query1 = sql.SQL('SELECT {fields} FROM {table}').format(
    fields=sql.SQL(',').join([
        sql.Identifier('broadcaster'),
        sql.Identifier('id')
    ]),
    table=sql.Identifier('broadcasters'))

cur.execute(query1)

broadcasters = dict(cur.fetchall())

query2 = sql.SQL('SELECT {fields} FROM {table}').format(
    fields=sql.SQL(',').join([
        sql.Identifier('reach'),
        sql.Identifier('id')
    ]),
    table=sql.Identifier('ad_reach'))

cur.execute(query2)

ad_reach = dict(cur.fetchall())


submediums['wydawca_nadawca'] = submediums['wydawca_nadawca'].map(broadcasters)
submediums['zasięg medium'] = submediums['zasięg medium'].map(ad_reach)


query3 = sql.SQL("INSERT INTO {table} ({field1}, {field2}, {field3}) VALUES (%s, %s ,%s)")

for index, submedium in submediums.iterrows():
    cur.execute(
        query3.format(
            table=sql.Identifier('mediums'),
            field1=sql.Identifier('submedium'),
            field2=sql.Identifier('broadcaster_id'),
            field3=sql.Identifier('ad_reach_id')), 
            (submedium["submedium"], submedium["wydawca_nadawca"], submedium["zasięg medium"],)
    )

conn.commit()

conn.close()

1 BET 12 3
2 BORING Włocławek 10 2
3 EMKA Głogów 2 1
4 EMKA Leszno 2 1
5 FORK FM 9 2
6 FORK FM Poznań (do 01.12.2013 RMKM Rock Poznań) 9 2
7 FROGGY WEATHER Bydgoszcz 8 2
8 FROGGY WEATHER Gdańsk 8 2
9 FROGGY WEATHER Katowice 8 2
10 FROGGY WEATHER Kraków 8 2
11 FROGGY WEATHER Opole 8 2
12 FROGGY WEATHER Poznań 8 2
13 FROGGY WEATHER Warszawa 8 2
14 FROGGY WEATHER Wrocław 8 1
15 GRAYSCALE Warszawa 3 2
16 HAM RADIO Kraków 8 2
17 HAM RADIO Opole 8 1
18 HAM RADIO Poznań 8 2
19 HAM RADIO Warszawa 8 2
20 KITTY Łódź 4 1
21 MINUS Gdańsk 12 4
22 MOOSHROOM 1 2
23 MUSICRADIO Katowice 12 2
24 MUSICRADIO Poznań 12 1
25 NOTRADIO 12 2
26 NOTRADIO Katowice 12 1
27 OLD 1 5 3
28 OLD 24 5 3
29 OLD 3 5 3
30 OLD Katowice 5 4
31 OLD Olsztyn 5 4
32 OLD PiK Bydgoszcz 5 4
33 OLD RDC Warszawa 5 4
34 OLD Radio Kraków 5 4
35 OLD Radio Poznań 5 4
36 OMG Białystok 6 1
37 PIANO RADIO Jelenia Góra 7 1
38 RADIO FIST 13 1
39 RADIO NO NO Toruń 11 2
40 RADIO NO NO Wrocław 11 2
41 RMKM Beskidy 9 2
42 RMKM Bełchatów 9 2
43 RM

In [7]:
submediums.head()

NameError: name 'submediums' is not defined

In [132]:
submediums.columns.values.tolist()

['submedium', 'wydawca_nadawca', 'zasięg medium']

In [54]:
ad_time = df[['data', 'godzina_bloku_reklamowego', 'gg', 'mm', 'dl_mod', 'daypart', 'dł_ujednolicona', 'kod_reklamy']]
ad_time['kod_reklamy'] = ad_time[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)
ad_time.index = ad_time.index + 1


conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

query1 = sql.SQL('SELECT {fields} FROM {table}').format(
    fields=sql.SQL(',').join([
        sql.Identifier('length'),
        sql.Identifier('id')
        ]),
    table=sql.Identifier('unified_lenghts'))

cur.execute(query1)
unified_lengths = dict(cur.fetchall())

query2 = sql.SQL('SELECT {fields} FROM {table}').format(
    fields=sql.SQL(',').join([
        sql.Identifier('daypart'),
        sql.Identifier('id')
        ]),
    table=sql.Identifier('dayparts'))

cur.execute(query2)
dayparts = dict(cur.fetchall())


ad_time['daypart'] = ad_time['daypart'].map(dayparts)
ad_time['dł_ujednolicona'] = ad_time['dł_ujednolicona'].map(unified_lengths)



query3 = sql.SQL(
    '''
    INSERT INTO {table} ({field1}, {field2}, {field3}, {field4}, {field5}, {field6}, {field7}, {field8}) 
    VALUES (%s, %s ,%s, %s ,%s, %s ,%s, %s)
    ''')

for _, elem in ad_time.iterrows():
    cur.execute(
        query3.format(
            table=sql.Identifier('ad_time_details'),
            field1=sql.Identifier('date'),
            field2=sql.Identifier('ad_slot_hour'),
            field3=sql.Identifier('gg'),
            field4=sql.Identifier('mm'),
            field5=sql.Identifier('length_mod'),
            field6=sql.Identifier('daypart_id'),
            field7=sql.Identifier('unified_length_id'),
            field8=sql.Identifier('ad_code')), 
            (elem['data'], elem['godzina_bloku_reklamowego'], elem['gg'], 
             elem['mm'], elem['dl_mod'], elem['daypart'], 
             elem['dł_ujednolicona'], elem['kod_reklamy'],
            )
    )

conn.commit()

conn.close()

ad_time.head(5)

/tmp/ipykernel_1418/410864604.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ad_time['kod_reklamy'] = ad_time[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)
/tmp/ipykernel_1418/410864604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_time['kod_reklamy'] = ad_time[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)
/tmp/ipykernel_1418/410864604.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,data,godzina_bloku_reklamowego,gg,mm,dl_mod,daypart,dł_ujednolicona,kod_reklamy
1,2023-08-01,8:00-8:29,8,20,29,1,3,2023-08-01 - 22194483
2,2023-08-01,10:30-10:59,10,59,14,2,1,2023-08-01 - 22179830
3,2023-08-01,15:00-15:29,15,22,30,2,3,2023-08-01 - 22194487
4,2023-08-01,14:30-14:59,14,42,29,2,3,2023-08-01 - 22194445
5,2023-08-01,13:30-13:59,13,58,30,2,3,2023-08-01 - 22194475


In [10]:
conn = psycopg2.connect(
    f'''dbname={tools.conf.DB} 
        user={tools.conf.USER} 
        host={tools.conf.HOST}
        port={tools.conf.PORT}
    '''
    )
cur = conn.cursor()

table = 'mediums'

query = sql.SQL(
    '''
    SELECT c.column_name 
    FROM information_schema.columns c 
    WHERE c.table_name = %s
    ORDER BY c.ordinal_position;
    ''').format()

cur.execute(query, (table,))

data = cur.fetchall()
temp = []
for elem in data[1:]:
    temp.append(elem[0])
data = temp

conn.close()

data

['submedium', 'broadcaster_id', 'ad_reach_id']

In [40]:
temp = df[['data', 'godzina_bloku_reklamowego', 'gg', 'mm', 'dl_mod', 'daypart', 'dł_ujednolicona', 'kod_reklamy']]
# temp['kod_reklamy'] = temp['kod_reklamy'].astype('object')
temp['kod_reklamy'] = temp[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)
temp.info()

/tmp/ipykernel_1418/3350514881.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp['kod_reklamy'] = temp[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366681 entries, 0 to 366680
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   data                       366681 non-null  object
 1   godzina_bloku_reklamowego  366681 non-null  object
 2   gg                         366681 non-null  int64 
 3   mm                         366681 non-null  int64 
 4   dl_mod                     366681 non-null  int64 
 5   daypart                    366681 non-null  object
 6   dł_ujednolicona            366681 non-null  object
 7   kod_reklamy                366681 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.4+ MB


/tmp/ipykernel_1418/3350514881.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['kod_reklamy'] = temp[['data', "kod_reklamy"]].apply(lambda x: str(x[0]) + ' - ' + str(x[1]), axis=1)
